In [ ]:
# 🚀 SETUP: Mount Drive and Install Dependencies
import os, subprocess, time, requests, threading, re
from google.colab import drive

# 🚀 Mount Google Drive and install system packages
from google.colab import drive
drive.mount('/content/drive')

!apt -y update && apt -y install -qq build-essential aria2
!pip install -q huggingface_hub

In [ ]:
import shutil

# Paths
drive_path = '/content/drive/MyDrive/koboldcpp_build'
local_path = '/content/koboldcpp'

# Build or restore KoboldCPP
if os.path.exists(drive_path) and os.path.exists(f'{drive_path}/koboldcpp_cublas.so'):
    print("Restoring KoboldCPP from Drive...")
    !mkdir -p {local_path}
    !cp -r {drive_path}/* {local_path}/
    print("✅ KoboldCPP restored!")
else:
    print("Building KoboldCPP from scratch...")
    !apt -y install -qq build-essential
    !git clone https://github.com/LostRuins/koboldcpp.git
    %cd koboldcpp
    !make LLAMA_CUBLAS=1 CUDA_DOCKER_ARCH=compute_80,code=sm_80
    !mkdir -p {drive_path}
    !cp -r /content/koboldcpp/* {drive_path}/
    print("✅ KoboldCPP built and saved to Drive!")

# Check GPU
!nvidia-smi


In [ ]:
# 📥 Download GGUF model
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="mradermacher/DeepSeek-R1-Distill-Qwen-32B-Uncensored-GGUF",
    filename="DeepSeek-R1-Distill-Qwen-32B-Uncensored.Q5_K_M.gguf")

print(f"✅ Model downloaded to: {model_path}")


In [ ]:
# 🎨 Set up Stable Diffusion WebUI with epicRealismXL from Drive
import os

%cd /content
if not os.path.exists("stable-diffusion-webui"):
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git

%cd stable-diffusion-webui

model_in_drive = "/content/drive/MyDrive/EpicRealism/epicrealismXL_vxviiCrystalclear.safetensors"
target_path = "models/Stable-diffusion/epicRealismXL.safetensors"
!mkdir -p models/Stable-diffusion

if os.path.exists(model_in_drive):
    if not os.path.exists(target_path):
        !ln -s "{model_in_drive}" "{target_path}"
        print("✅ Model linked from Drive")
    else:
        print("🔁 Model already linked")
else:
    print("❌ Model not found in Drive")


In [ ]:
# 📥 Download cloudflared
!wget -q -O /content/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x /content/cloudflared

In [ ]:
# 🚀 Launch Stable Diffusion WebUI in background
import subprocess, time, requests

%cd /content/stable-diffusion-webui

webui_cmd = [
    "python", "launch.py", "--deepdanbooru",
    "--api", "--listen", "--port", "7860",
    "--skip-torch-cuda-test", "--no-half-vae",
    "--ckpt", "models/Stable-diffusion/epicRealismXL.safetensors"
]

subprocess.Popen(webui_cmd)
print("✅ WebUI launched in background. Give it 60–90 seconds to initialize.")

# Optional: wait for API to become available
print("⏳ Waiting for Stable Diffusion API...")
for i in range(600):
    try:
        r = requests.get("http://127.0.0.1:7860/sdapi/v1/sd-models", timeout=5)
        if r.status_code == 200:
            print("✅ Stable Diffusion API is ready!")
            break
    except:
        pass
    if i % 30 == 0 and i > 0:
        print(f"Still waiting... ({i}s)")
    time.sleep(1)
else:
    print("❌ Stable Diffusion API did not respond in time.")


In [ ]:
# 🌐 Start Cloudflare tunnel to expose WebUI
import subprocess, re

tunnel_cmd = "/content/cloudflared tunnel --url http://localhost:7860 --no-autoupdate"

print("⏳ Starting Cloudflare tunnel for WebUI...")
tunnel_proc = subprocess.Popen(tunnel_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Print the tunnel URL when it appears
for line in tunnel_proc.stdout:
    print(line.strip())
    match = re.search(r'https://[a-zA-Z0-9\-]+\.trycloudflare\.com', line)
    if match:
        print(f"\n🌐 Public WebUI URL: {match.group(0)}")
        break

In [ ]:
%cd /content/koboldcpp

!python3 koboldcpp.py \
  --model {model_path} \
  --port 5001 \
  --host 0.0.0.0 \
  --remotetunnel \
  --threads 4 \
  --usecublas \
  --gpulayers 32 \
  --contextsize 4096